# Math 5 - Linear Algebra: Systems of linear equations

In [5]:
import numpy as np
import scipy.linalg

## 1. Systems of linear equations
We have already covered Gaussian Elimination and it's application to find the inverse of a matrix. Another application is **solving systems of linear equations**. Let's suppose we have the following system of two linear equations.

$$
x_1 - 2x_2 = 1 \\
3x_1 + 2x_2 = 11
$$

The system can be written in matrix form $Ax = b$. Where $A$ is the coefficient matrix, $x$ is the vector with the unknowns and $b$ is the right hand side vector.

$$
\begin{bmatrix}1 & -2 \\ 3 & 2\end{bmatrix}\begin{bmatrix}x_1 \\ x_2\end{bmatrix} = \begin{bmatrix}1 \\ 11\end{bmatrix} 
$$

The right hand side can then be added to the matrix as a new column. When we perform a row reduction for the matrix this affects also the vector $b$ (the same idea as for the inverse calculation).

$$
\begin{bmatrix}1 & -2  & 1 \\ 3 & 2 & 11\end{bmatrix} \rightarrow \begin{bmatrix}1 & -2  & 1 \\ 0 & 8 & 8\end{bmatrix}
\rightarrow \begin{bmatrix}1 & -2 & 1 \\ 0 & 1 & 1\end{bmatrix}
\rightarrow \begin{bmatrix}1 & 0 & 3 \\ 0 & 1 & 1\end{bmatrix}
$$

We are able to solve the system with the reduced augmented matrix in a **backsubstitution** step. In this example we can read the solution directly from the matrix.

$$
x_2 = 1, x_1 = 3
$$

In [2]:
augmented = np.array([[1, -2, 1], [3, 2, 11]])
augmented

array([[ 1, -2,  1],
       [ 3,  2, 11]])

The already known function `rref`is used again ;-)

In [3]:
def rref(Mat):
    # Calculates the row-reduced-echelon-form for the given matrix
    # from https://rosettacode.org/wiki/Reduced_row_echelon_form
    M = np.copy(Mat)
    lead = 0
    row_count = len(M)
    col_count = len(M[0])
    for r in range(row_count):
        if col_count <= lead:
            return M
        i = r
        while M[i][lead] == 0:
            i += 1
            if row_count == i:
                i = r
                lead += 1
                if col_count == lead:
                    return M
        M[i], M[r] = M[r], M[i]
        if not M[r][lead] == 0:
            M[r] = M[r] / M[r][lead]
        for i in range(row_count):
            if not i == r:
                lv = M[i][lead]
                M[i] = [iv - lv*rv for rv,iv in zip(M[r], M[i])]
        lead += 1
    return M

We find the same solution as above with the `rref` function.

In [4]:
rref(augmented)

array([[1, 0, 3],
       [0, 1, 1]])

## 2. LU decomposition
The Gaussian Elimination let us solve a system of linear equations for a single vector $b$. But, we have to repeat the whole process for any different right hand side. This is not very efficient. Therefore, we would like to avoid repeating all elimination steps over and over again. This can be done with the **LU decomposition**. Thus a matrix is written as a product of a lower- and a upper-triangular matrix.

_Note: A triangular matrix is a special case for square matrices._
$$
L_2 = \begin{bmatrix}a_{11} & 0 \\ a_{21} & a_{22}\end{bmatrix}, 
U_2 = \begin{bmatrix}a_{11} & a_{12} \\ 0 & a_{22}\end{bmatrix}
$$

Finding these two matrices is done with a series of elimination steps. These steps are then recorded in the two matrices.

$$
Ax = b = (LU)x
$$

Usually in math programs the LU decomposition is implemented with pivoting. Then we have a third matrix $P$.

$$
PA = LU
$$

This is also the case in the `lu` function of the `scipy.linalg` module which we are using in this notebook.

In [6]:
A = np.array([[1, -2], [3, 2]])
P, L, U = scipy.linalg.lu(A)
A

array([[ 1, -2],
       [ 3,  2]])

When we multiply these three matrices we got the matrix `A` again.

In [7]:
P.dot(L.dot(U))

array([[ 1., -2.],
       [ 3.,  2.]])

### 2.1 Solve equations with LU decomposition
To solve a system of linear equations with the LU decomposition we first multiply both sides with the pivot matrix.

$$
PAx = Pb = c \equiv LUx
$$

In one forward substitution step we get $y$. We define $Ux = y$.

$$
Ly = c
$$

With $y$ we are able to find $x$ in one back substitution step.

$$
Ux = y
$$

Very nice! Let's do an example.

In [11]:
A = np.random.rand(4,4)
b = np.random.rand(4,1)
P, L, U = scipy.linalg.lu(A)

Then we calculate $x$ and $y$. We use the function `np.linalg.solve`. This function does not compute the inverse. Instead it uses an optimized routine with **forward** and **backward substitution**.

In [12]:
d = P.dot(b)
y = np.linalg.solve(L, d)
x = np.linalg.solve(U, y)

Then we verify the result!

In [13]:
np.allclose(A.dot(x), b)

True

Of course we can use the `solve` function directly.

In [14]:
x = np.linalg.solve(A, b)
np.allclose(A.dot(x), b)

True

#### Solve with inverse matrices
Instead we can do the calculation with the inverse of $L$ and $U$. Thats simple for these small matrices.

In [17]:
y = np.linalg.inv(L).dot(d)
x = np.linalg.inv(U).dot(y)

np.allclose(A.dot(x), b)

True

#### Different right hand side
For a **different** right hand side we just repeat the steps for the new result vector. The LU decomposition can be **reused**.

In [20]:
b = np.random.rand(4,1)
d = P.dot(b)
y = np.linalg.solve(L, d)
x = np.linalg.solve(U, y)
np.allclose(A.dot(x), b)

True